In [ ]:
import torch

# Check if a GPU is available
print("GPU Available:", torch.cuda.is_available())

# Check the name of the GPU
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


In [ ]:
from fastai.vision.all import *
import matplotlib.pyplot as plt

### Data Loader
path = Path('../Spectrogram_Images')

dls = ImageDataLoaders.from_folder(
    path,
    train = '.',
    valid_pct=0.2,  # 20% of data for validation
    item_tfms = Resize(224), # Resize images to 224x224 pixels for CNN
    batch_tfms=aug_transforms(mult=1.0)  # Apply basic augmentations
)

# Show a batch of images to verify data loading
dls.show_batch(max_n=9, figsize=(8, 8))

### CNN
from fastai.callback.tracker import EarlyStoppingCallback
learn = vision_learner(dls, resnet34, metrics=accuracy,wd=1e-4, cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3)])



lr_steep = learn.lr_find().valley

plt.show()

learn.model[-1].add_module('dropout', nn.Dropout(p=0.5))
learn.fine_tune(10, base_lr=lr_steep)

# Fine-tune with early stopping

# Plot training and validation loss curves
learn.recorder.plot_loss()




In [4]:
### CNN
from fastai.vision.all import *
from fastai.callback.tracker import EarlyStoppingCallback
path = Path('../Spectrogram_Images/s')
set_seed(31)
dls = ImageDataLoaders.from_folder(
    path,
    train = '.',
    valid_pct=0.2,  # 20% of data for validation
    item_tfms = Resize(224), # Resize images to 224x224 pixels for CNN
    batch_tfms=aug_transforms(mult=1.0)  # Apply basic augmentations
)
learn = vision_learner(dls, resnet34, metrics=accuracy, wd=1e-4 ,cbs=[EarlyStoppingCallback(monitor='valid_loss', patience=3)])

learn.load('final_model')
learn.model.eval()
# Validate the loaded model on the validation dataset
validation_results = learn.validate(dl=dls.valid)
print(f"Validation Results: {validation_results}")

# Print the validation results
validation_loss, validation_accuracy = validation_results
print(f"Validation Loss: {validation_loss:.4f}")
print(f"Validation Accuracy: {validation_accuracy:.4f}")

# Compare with training metrics (if available)
training_results = learn.validate(dl=dls.train)
training_loss, training_accuracy = training_results
print(f"Training Loss: {training_loss:.4f}")
print(f"Training Accuracy: {training_accuracy:.4f}")

# Check for overfitting
if training_accuracy > validation_accuracy + 0.05:
    print("Warning: The model might be overfitting.")
else:
    print("The model seems to be generalizing well.")
    


Validation Results: [0.3122069239616394, 0.8672986030578613]
Validation Loss: 0.3122
Validation Accuracy: 0.8673


Training Loss: 0.4510
Training Accuracy: 0.8065
The model seems to be generalizing well.


In [16]:
from sklearn.metrics import classification_report


path = Path("Test")

learn = load_learner('Test/final_model.pkl')

test_dl = learn.dls.test_dl(get_image_files(path))

preds, targets  =learn.get_preds(dl=dls.train)

# Check if preds or targets are None
if preds is None or targets is None:
    raise ValueError("Predictions or targets are None. Please check the DataLoader and the test data.")

# Print the shapes of preds and targets for debugging
print(f"Predictions shape: {preds.shape}")
print(f"Targets shape: {targets.shape}")

# Get predictions and targets
preds, targets = learn.get_preds(dl=test_dl)

# Check if preds or targets are None
if preds is None or targets is None:
    raise ValueError("Predictions or targets are None. Please check the DataLoader and the test data.")

# Print the shapes of preds and targets for debugging
print(f"Predictions shape: {preds.shape}")
print(f"Targets shape: {targets.shape}")

# Calculate accuracy
acc = accuracy(preds, targets)
print(f'Accuracy on test set: {acc.item():.4f}')

# Plot confusion matrix
interp = ClassificationInterpretation.from_learner(learn, dl=test_dl)
interp.plot_confusion_matrix()
plt.show()

# Print classification report
y_pred = preds.argmax(dim=1)
y_true = targets
print(classification_report(y_true, y_pred, target_names=learn.dls.vocab))

# Print additional information
print(f"Number of classes: {len(learn.dls.vocab)}")
print(f"Class names: {learn.dls.vocab}")
print(f"Number of test images: {len(test_dl.dataset)}")

Predictions shape: torch.Size([848, 2])
Targets shape: torch.Size([848])


ValueError: Predictions or targets are None. Please check the DataLoader and the test data.